<a href="https://colab.research.google.com/github/kyotoman-koshida/bokete/blob/main/notebooks/20220929.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://qiita.com/toshiyuki_tsutsui/items/a01e2a3ffae035ef644c

boketeのお題画像は、画像の時点で既に面白い気がする。それに対して、ImageNetの画像はカテゴリの意識を強く感じる気がする。だから、ImageNetで学習ずみのモデルは、boketeの画像に対して予測しきれないのではないか。

GPUの制限もあるので、ここではまずお試しとして、1番軽いMobileNetの学習ずみモデルを使って転移学習を試みる！

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers[ja]
!pip install --quiet sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 38.4 MB/s 
     |████████████████████████████████| 163 kB 57.2 MB/s 
     |████████████████████████████████| 47.4 MB 48.6 MB/s 
     |████████████████████████████████| 583 kB 53.6 MB/s 
     |████████████████████████████████| 13.4 MB 36.3 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=bd07674fb60a9c97411b318fd9b10abec5cb4a951bb1b69110c2edba40fb5f90
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7426 sha256=e2b02ee8ec4d93165a935187bbdc06e2974efefdfbe11f6eb7a261a28a6fdf81
  Stored in directory: /root/.cache/pip/wheels/ce/4d/f1/170bb74b559ca338113c0315c9805e16dfd0a12411ec6b1122
  Created wheel for unidic-lite: fil

In [ ]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import os
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from scipy.special import softmax

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset

from transformers import (
    AutoTokenizer, AutoModel, MMBTForClassification, MMBTConfig, AutoConfig,
    Trainer, TrainingArguments,
)
import transformers

from torchvision.io import read_image
from torchvision.models import ResNet152_Weights, resnet152
from torchvision.models import MobileNet_V2_Weights, mobilenet_v2

from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

seed_everything(42)

MMBT部分

In [ ]:
pretrained_weight = MobileNet_V2_Weights.IMAGENET1K_V2
model = mobilenet_v2(weights=pretrained_weight)
modules = list(model.children())[:-1]
model = nn.Sequential(*modules)
print(model)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

Sequential(
  (0): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, m

学習ずみMobileNetをダウンロード

In [ ]:
# 学習済みモデルを改造する関数
def modify_pretrain_MobileNet(pretrained_weight):
    model = mobilenet_v2(weights=pretrained_weight)
    modules = list(model.children())[:-1]
    # modules.append(torch.nn.Dropout(0.2))
    modules.append(torch.nn.Conv2d(1280, 2048, kernel_size=(1,1), stride=(1, 1), bias=False))
    modules.append(torch.nn.BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
    # modules[-1][-1] = torch.nn.Linear(1280, 2048)
    # modules.append(torch.nn.Linear(1280, 2048))
    # model.fc = torch.nn.Linear(1280, 2048)
    # modules.append(torch.nn.Linear(2048, 1000))
    # model.fc = torch.nn.Linear(2048, 1000)
    model = nn.Sequential(*modules)
    return model

# 画像データをEmbeddingしていきます
class ImageEncoder(nn.Module):
    POOLING_BREAKDOWN = {1: (1, 1), 2: (2, 1), 3: (3, 1), 4: (2, 2), 5: (5, 1), 6: (3, 2), 7: (7, 1), 8: (4, 2), 9: (3, 3)}
    def __init__(self, pretrained_weight):
        super().__init__()
        # model = mobilenet_v2(weights=pretrained_weight)# ResNet152からMobileNetに変更
        model = modify_pretrain_MobileNet(pretrained_weight)
        # modules = list(model.children())[:-1]
        modules = list(model.children())
        self.model = nn.Sequential(*modules)
        #self.model.fc = torch.nn.Linear(224, 2048)
        # self.model.fc = torch.nn.Linear(3, 2048)
        # self.model.fc = torch.nn.Linear(1280, 2048)
        self.pool = nn.AdaptiveAvgPool2d(self.POOLING_BREAKDOWN[3])

    def forward(self,  x):
        # print("x.shape:",x.shape)
        # print("self.model(x).size:",self.model(x).size())
        out = self.pool(self.model(x))
        # print("self.pool:",self.pool)
        # print("out = self.pool(self.model(x))")
        # print(out.size())
        out = torch.flatten(out, start_dim=2)
        # print("out = torch.flatten(out, start_dim=2)")
        # print(out.size())
        out = out.transpose(1, 2).contiguous()
        # print("out = out.transpose(1, 2).contiguous()")
        # print(out.size())
        return out

In [ ]:
def read_jpg(path):
    image_tensor = read_image(path)
    if image_tensor.shape[0] == 1:
        # 1channel=白黒画像があるので3channelにconvertしています。
        image_tensor = image_tensor.expand(3, *image_tensor.shape[1:])
    return image_tensor

class BoketeTextImageDataset(Dataset):
    def __init__(self, df, tokenizer, max_seq_len:int, image_transform):
        self.df = df
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        self.image_transforms = image_transform.transforms()

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        sentence = torch.tensor(self.tokenizer.encode(row["text"], max_length=self.max_seq_len, padding="max_length", truncation=True))
        start_token, sentence, end_token = sentence[0], sentence[1:-1], sentence[-1]
        sentence = sentence[:self.max_seq_len]

        image = self.image_transforms(read_jpg(row["img_path"]))

        return {
            "image_start_token": start_token,
            "image_end_token": end_token,
            "sentence": sentence,
            "image": image,
            "label": torch.tensor(row["is_laugh"]),
        }

def collate_fn(batch):
    lens = [len(row["sentence"]) for row in batch]
    bsz, max_seq_len = len(batch), max(lens)

    mask_tensor = torch.zeros(bsz, max_seq_len, dtype=torch.long)
    text_tensor = torch.zeros(bsz, max_seq_len, dtype=torch.long)

    for i_batch, (input_row, length) in enumerate(zip(batch, lens)):
        text_tensor[i_batch, :length] = input_row["sentence"]
        mask_tensor[i_batch, :length] = 1

    img_tensor = torch.stack([row["image"] for row in batch])
    tgt_tensor = torch.stack([row["label"] for row in batch])
    img_start_token = torch.stack([row["image_start_token"] for row in batch])
    img_end_token = torch.stack([row["image_end_token"] for row in batch])

    return {
        "input_ids":text_tensor,
        "attention_mask":mask_tensor,
        "input_modal":img_tensor,
        "modal_start_tokens":img_start_token,
        "modal_end_tokens":img_end_token,
        "labels":tgt_tensor,
    }

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

学習データの格納部分

In [ ]:
INPUT = "/content/drive/MyDrive/会社/Nishika/bokete" # 所望のディレクトリに変更してください。
train_image_path = "/content/drive/MyDrive/会社/Nishika/bokete/content/train/"
test_image_path = "/content/drive/MyDrive/会社/Nishika/bokete/content/test/"

train_df = pd.read_csv(os.path.join(INPUT, "train.csv"))
test_df = pd.read_csv(os.path.join(INPUT, "test.csv"))
submission_df = pd.read_csv(os.path.join(INPUT, "sample_submission.csv"))

train_df["img_path"] = train_image_path + train_df["odai_photo_file_name"]
test_df["img_path"] = test_image_path + test_df["odai_photo_file_name"]

In [ ]:
test_df["is_laugh"] = 0

データの分割部分

In [ ]:
trn_idx, val_idx = train_test_split(list(range(len(train_df))), test_size=0.2, random_state=42, stratify=train_df["is_laugh"])

In [ ]:
trn_ds = BoketeTextImageDataset(train_df.iloc[trn_idx], tokenizer, 48, image_transform=MobileNet_V2_Weights.IMAGENET1K_V2)
val_ds = BoketeTextImageDataset(train_df.iloc[val_idx], tokenizer, 48, image_transform=MobileNet_V2_Weights.IMAGENET1K_V2)

In [ ]:
test_ds = BoketeTextImageDataset(test_df, tokenizer, 48, image_transform=MobileNet_V2_Weights.IMAGENET1K_V2)

In [ ]:
transformer_config = AutoConfig.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
transformer = AutoModel.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
transformer

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
config = MMBTConfig(transformer_config, num_labels=2)
model = MMBTForClassification(config, transformer, ImageEncoder(MobileNet_V2_Weights.IMAGENET1K_V2))

In [ ]:
config

In [ ]:
model

MMBTForClassification(
  (mmbt): MMBTModel(
    (transformer): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(32000, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
   

In [ ]:
config

In [ ]:
config.use_return_dict = True

In [ ]:
model.config = model.mmbt.config

In [ ]:
trainer_args = TrainingArguments(
    output_dir="/content/mmbt_exp01",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    logging_steps=50,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=12,
    save_total_limit=1,
    # fp16=True,#GPUでのみ使う
    remove_unused_columns=False,
    gradient_accumulation_steps=20,
    load_best_model_at_end=True,
    logging_dir='./logs',
    report_to="none"
)

In [ ]:
model

MMBTForClassification(
  (mmbt): MMBTModel(
    (transformer): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(32000, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
   

In [ ]:
trainer = Trainer(
    model=model,
    args=trainer_args,
    tokenizer=tokenizer,
    train_dataset=trn_ds,
    eval_dataset=val_ds,
    data_collator=collate_fn,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 19969
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 160
  Gradient Accumulation steps = 20
  Total optimization steps = 372


Step,Training Loss,Validation Loss


In [ ]:
val_preds = trainer.predict(val_ds).predictions

In [ ]:
# sanity check
log_loss(val_ds.df["is_laugh"].values, softmax(val_preds, axis=-1))

In [ ]:
accuracy_score(val_ds.df["is_laugh"].values, np.argmax(val_preds, axis=-1))

予測

In [ ]:
preds = trainer.predict(test_ds).predictions

In [ ]:
submission_df["is_laugh"] = softmax(preds, axis=-1)[:, 1]

In [ ]:
submission_df["is_laugh"] = submission_df["is_laugh"].astype(float)

In [ ]:
OUTPUT = "/content/drive/MyDrive/会社/Nishika/bokete/results" # ディレクトリを指定してください
submission_df.to_csv(os.path.join(OUTPUT,'20220929MobileNet_MMBT.csv'), index=False)